<a href="https://colab.research.google.com/github/mboukabous/pytorch_scripts/blob/main/05_pytorch_scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Create a script directory
import os
os.makedirs("scripts")

## 1. Get data

In [ ]:
%%writefile scripts/get_data.py
"""
Setup train and test directories for images.
"""
import os
import zipfile
from typing import Tuple

from pathlib import Path
import requests

def download_data(web_link: str) -> Tuple[Path, Path]:
  """
  Download data from the internet as zip file and extract it
  to get train and test paths.

  Args:
    web_link: dataset url in the web (raw zip if using github links)

  Returns:
    A tuple of (train_path, test_path)

  Example usage:
    train_dir, test_dir = download_data(web_link=data_link)
  """

  # Setup path to data folder
  data_path = Path("data/")
  zip_name = web_link.split("/")[-1]
  data_name = zip_name.split(".")[0]

  zip_path = data_path / zip_name
  image_path = data_path / data_name

  # If the image folder doesn't exist, download it and prepare it...
  if image_path.is_dir():
      print(f"{image_path} directory exists.")
  else:
      print(f"Did not find {image_path} directory, creating one...")
      image_path.mkdir(parents=True, exist_ok=True)

  # Download web data
  with open(zip_path, "wb") as f:
      request = requests.get(web_link)
      print(f"Downloading {data_name} data...")
      f.write(request.content)

  # Unzip pizza, steak, sushi data
  with zipfile.ZipFile(zip_path, "r") as zip_ref:
      print(f"Unzipping {data_name} data...")
      zip_ref.extractall(image_path)

  # Remove zip file
  os.remove(zip_path)

  # Setup train and testing paths
  train_dir = image_path / "train"
  test_dir = image_path / "test"

  return train_dir, test_dir

def local_data(local_link: str) -> Tuple[Path, Path]:
  """
  Get train and test paths from a local dataset

  Args:
    local_link: dataset local url

  Returns:
    A tuple of (train_path, test_path)

  Example usage:
    train_dir, test_dir = local_data(local_link=data_link)
  """

  # Setup path to data folder
  image_path = Path(local_link)

  # If the image folder doesn't exist, raise an error
  assert not image_path.is_dir(), "Dataset doesn't exist, please enter a correct url"

  # Setup train and testing paths
  train_dir = image_path / "train"
  test_dir = image_path / "test"

  return train_dir, test_dir

Writing scripts/get_data.py


## 2. Create Datasets and DataLoaders

In [ ]:
%%writefile scripts/data_setup.py
"""
Contains functionality for creating PyTorch DataLoader's for
Image classification data.
"""
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(train_dir: str, test_dir: str, batch_size: int,
                       train_transform: transforms.Compose=None, test_transform: transforms.Compose=None,
                       img_size: tuple=None, num_workers: int=NUM_WORKERS):
  """
  Creates training and testing DataLoders
  Takes in training and testing dir paths with other params
  And turns them into PyTorch Dataset and DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    batch_size: Number of samples per batch in each DataLoader.
    train_transform: transforms to perform on training data.
    test_transform: transforms to perform on testing data.
    img_size: a tuple used to create a simple transform in case train_transform or test_transform is None.
    num_workers: An integer for number of workers per DataLoader (default is CPU number available).

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).

  Example usage:
    train_dataloader, test_dataloader, class_names = create_dataloaders(
      train_dir=path/to/train_dir, test_dir=path/to/test_dir,
      train_transform=some_transforms_compose, test_transform=some_transforms_compose,
      batch_size=32, num_workers=4)
  """
  # Check is a transform exist, otherwise create a simple one using img_size
  if not train_transform:
    train_transform = transforms.Compose([
      transforms.Resize(img_size),
      transforms.ToTensor()
    ])
  if not test_transform:
    test_transform = transforms.Compose([
      transforms.Resize(img_size),
      transforms.ToTensor()
    ])

  # Use ImageFolder to create dataset(s)
  train_data = datasets.ImageFolder(root=train_dir, # target folder of images
                                    transform=train_transform, # transforms to perform on data (images)
                                    target_transform=None) # transforms to perform on labels (if necessary)
  test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)

  # Get the class_names
  class_names = train_data.classes

  # Turn images into DataLoaders
  train_dataloader = DataLoader(dataset=train_data,
                                batch_size=batch_size, # how many samples per batch?
                                num_workers=num_workers, # how many subprocesses to use for data loading?
                                shuffle=True, # shuffle the data?
                                pin_memory=True) # enable fast data transfer to CUDA-GPU
  test_dataloader = DataLoader(dataset=test_data,
                               batch_size=batch_size,
                               num_workers=num_workers,
                               shuffle=False, # don't usually need to shuffle testing data
                               pin_memory=True)

  return train_dataloader, test_dataloader, class_names

Writing scripts/data_setup.py


## 3. Making a model (TinyVGG)

TinyVGG from the CNN Explainer website.

In [ ]:
%%writefile scripts/model_builder.py
"""
Contains PyTorch model code to instantiate a
TinyVGG model.
"""

import torch
from torch import nn

class TinyVGG(nn.Module):
  """Creates the TinyVGG architecture.

  Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/

  Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
  """
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
      super().__init__()
      self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape,
                    out_channels=hidden_units,
                    kernel_size=3, # how big is the square that's going over the image?
                    stride=1, # default
                    padding=0), # options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2) # default stride value is same as kernel_size
      )
      self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
      )
      self.classifier = nn.Sequential(
          nn.Flatten(),
          # Where did this in_features shape come from?
          # It's because each layer of our network compresses and changes the shape of our inputs data.
          nn.Linear(in_features=hidden_units*13*13, out_features=output_shape)
      )

  def forward(self, x: torch.Tensor):
      """x = self.conv_block_1(x)
      x = self.conv_block_2(x)
      x = self.classifier(x)
      return x"""
      return self.classifier(self.conv_block_2(self.conv_block_1(x))) # <- leverage the benefits of operator fusion

Writing scripts/model_builder.py


## 4. Creating `train_step()` and `test_step()` functions and `train()` to combine them  

In [ ]:
%%writefile scripts/engine.py
"""
Contains functions for training and testing a PyTorch model.
"""

from typing import Tuple, Dict, List
from tqdm.auto import tqdm
import torch

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  """Trains a PyTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward
  pass, loss calculation, optimizer step).

  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
  """
  # Put model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader data batches
  for batch, (X, y) in enumerate(dataloader):
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      y_pred = model(X)

      # 2. Calculate  and accumulate loss
      loss = loss_fn(y_pred, y)
      train_loss += loss.item()

      # 3. Optimizer zero grad
      optimizer.zero_grad()

      # 4. Loss backward
      loss.backward()

      # 5. Optimizer step
      optimizer.step()

      # Calculate and accumulate accuracy metric across all batches
      y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
      train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
  """
  # Put model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager
  with torch.inference_mode():
      # Loop through DataLoader batches
      for batch, (X, y) in enumerate(dataloader):
          # Send data to target device
          X, y = X.to(device), y.to(device)

          # 1. Forward pass
          test_pred_logits = model(X)

          # 2. Calculate and accumulate loss
          loss = loss_fn(test_pred_logits, y)
          test_loss += loss.item()

          # Calculate and accumulate accuracy
          test_pred_labels = test_pred_logits.argmax(dim=1)
          test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  # Adjust metrics to get average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List[float]]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
  """
  # Create empty results dictionary
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }

  # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
      train_loss, train_acc = train_step(model=model,
                                         dataloader=train_dataloader,
                                         loss_fn=loss_fn,
                                         optimizer=optimizer,
                                         device=device)
      test_loss, test_acc = test_step(model=model,
                                      dataloader=test_dataloader,
                                      loss_fn=loss_fn,
                                      device=device)

      # Print out what's happening
      print(
          f" Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
      )

      # Update results dictionary
      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)

  # Return the filled results at the end of the epochs
  return results

Writing scripts/engine.py


## 5. Creating a function to save the model and it's results

In [ ]:
%%writefile scripts/utils.py
"""
Contains various utility functions for PyTorch model training.
"""

import torch
from pathlib import Path
import pandas as pd

def save_model(model: torch.nn.Module, target_dir: str, model_name: str):
  """
  Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True, exist_ok=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(), f=model_save_path)

def save_results(results: dict, target_dir: str, model_name: str):
  """Saves a PyTorch model results to a target directory.

  Args:
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model results.

  Example usage:
    save_model(model=model_name,
               target_dir="results",
               model_name="model_name_results")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True, exist_ok=True)

  # Create model results save path
  model_save_path = target_dir_path / model_name

  # Creating a dataframe from results dictionnary
  results_df = pd.DataFrame(results)

  # Save the model state_dict()
  print(f"[INFO] Saving results to: {model_save_path}")
  results_df.to_csv(model_save_path, index=False)

def load_model(model: torch.nn.Module, model_path: str) -> torch.nn.Module:
  """
  Load a PyTorch model parameters from a target path.

  Args:
    model_path: A path for loading the model from.

  Example usage:
    load_model(model_path="models/model_name.pth")
  """
  # Load the model state_dict()
  print(f"[INFO] Loading model from: {model_path}")
  model.load_state_dict(torch.load(model_path, weights_only=False))
  return model

Writing scripts/utils.py


## 6. Train, evaluate and save the model

In [ ]:
%%writefile train.py
"""
Trains a PyTorch image classification model.
"""

# Importations
import argparse
import os
import torch
from torch import nn
from torchvision import transforms
from timeit import default_timer as timer
import random
from scripts import data_setup, engine, model_builder, utils, get_data

# Setup Parser
parser = argparse.ArgumentParser(prog='Model training', description='Train a model on custom data.')
parser.add_argument('--epochs', type=int, default=10)
parser.add_argument('--batch_size', type=int, default=32)
parser.add_argument('--color_channels', type=int, default=3)
parser.add_argument('--hidden_units', type=int, default=10)
parser.add_argument('--lr', type=float, default=0.001)
parser.add_argument('--img_size', type=int, default=64)

parser.add_argument('--local', action='store_true')
parser.add_argument('--url', type=str, default="data/dataset")

# Setup HyperParameters
NUM_EPOCHS = parser.parse_args().epochs
BATCH_SIZE = parser.parse_args().batch_size
INPUT_SHAPE = parser.parse_args().color_channels
HIDDEN_UNITS = parser.parse_args().hidden_units
LEARNING_RATE = parser.parse_args().lr
IMG_SIZE = (parser.parse_args().img_size,parser.parse_args().img_size)

IS_LOCAL = parser.parse_args().local
URL = parser.parse_args().url

# Setup directories
if IS_LOCAL:
  train_dir, test_dir = get_data.local_data(URL)
else:
  train_dir, test_dir = get_data.download_data(URL)

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create or import transforms here

# Create dataLoaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               batch_size=BATCH_SIZE,
                                                                               img_size=IMG_SIZE)
                                                                               #train_transform=data_transform
                                                                               #test_transform=data_transform

# Recreate an instance of TinyVGG
model = model_builder.TinyVGG(input_shape=INPUT_SHAPE,
                              hidden_units=HIDDEN_UNITS,
                              output_shape=len(class_names)).to(device)

# Setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

# Start the timer
start_time = timer()

# Train model using engine.py
model_results = engine.train(model=model,
                             train_dataloader=train_dataloader,
                             test_dataloader=test_dataloader,
                             loss_fn=loss_fn,
                             optimizer=optimizer,
                             epochs=NUM_EPOCHS,
                             device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

# Save the model using utils.py
models_dir = "models"
results_dir = "results"
model_name = f"{model.__class__.__name__}_model_{NUM_EPOCHS}_ep_{LEARNING_RATE}_lr_{BATCH_SIZE}_bs_{HIDDEN_UNITS}_hu_{INPUT_SHAPE}_cc_{parser.parse_args().img_size}_img#{random.randint(1, 100)}"

utils.save_model(model=model, target_dir=models_dir, model_name=model_name+".pth")
utils.save_results(results=model_results, target_dir=results_dir, model_name=model_name+".csv")

Writing train.py


## 7. Test a model

In [ ]:
%%writefile test.py
"""
Test a PyTorch classification model
"""

#Importations
import argparse
import os
import torch
from torch import nn
from torchvision import transforms
from timeit import default_timer as timer
import random
from tqdm.auto import tqdm
from scripts import data_setup, model_builder, utils, get_data

# Setup Parser
parser = argparse.ArgumentParser(prog='Model testing', description='Test a model on custom data.')

parser.add_argument('--model', type=str, default="models/model.pth")
parser.add_argument('--local', action='store_true')
parser.add_argument('--url', type=str, default="data/dataset")

# Setup HyperParameters
MODEL_PATH = parser.parse_args().model
IS_LOCAL = parser.parse_args().local
URL = parser.parse_args().url

# Get HyperParameters
BATCH_SIZE = int(MODEL_PATH.split("_bs_")[0].split("_")[-1])
HIDDEN_UNITS = int(MODEL_PATH.split("_hu")[0].split("_")[-1])
IMG_SIZE = (int(MODEL_PATH.split("_img")[0].split("_")[-1]), int(MODEL_PATH.split("_img")[0].split("_")[-1]))
INPUT_SHAPE = int(MODEL_PATH.split("_cc_")[0].split("_")[-1])

# Setup directories
if IS_LOCAL:
  train_dir, test_dir = get_data.local_data(URL)
else:
  train_dir, test_dir = get_data.download_data(URL)

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create or import transforms here

# Create dataLoaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               batch_size=BATCH_SIZE,
                                                                               img_size=IMG_SIZE)
                                                                               #train_transform=data_transform
                                                                               #test_transform=data_transform

# Recreate an instance of TinyVGG
model = model_builder.TinyVGG(input_shape=INPUT_SHAPE,
                              hidden_units=HIDDEN_UNITS,
                              output_shape=len(class_names)).to(device)

# Setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss()

# Load the model parameters
model = utils.load_model(model=model, model_path=MODEL_PATH)

# Test loop
test_time_start = timer()
loss, acc = 0, 0
model.eval()

with torch.inference_mode():
  for X, y in tqdm(test_dataloader):
    # Device agnostic code
    X = X.to(device)
    y = y.to(device)

    # Forward pass and calculate the metrics
    y_pred = model(X)
    loss += loss_fn(y_pred, y)

    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    acc += (y_pred_class == y).sum().item()/len(y_pred)

  # calculate total loss and accuracy (average)
  loss /= len(test_dataloader)
  acc /= len(test_dataloader)

# End the timer and print out how long it took
test_time_end = timer()

# Print results
print(f"[INFO] Model: {model.__class__.__name__}")
print(f"[INFO] Model loss: {loss:.4f}")
print(f"[INFO] Model accuracy: {acc*100:.4f}%")
print(f"[INFO] Model execution time: {test_time_end-test_time_start:.3f} seconds")

Writing test.py


## 8. Predict on custom data

In [ ]:
%%writefile predict.py
"""
Makes a prediction on a target image and plots the image
with its prediction using a PyTorch model.
"""

# Importations
import argparse
from pathlib import Path
import requests
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import transforms
from scripts import model_builder, utils

# Device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# Setup Parser
parser = argparse.ArgumentParser(prog='Model predicting', description='Prediction using a PyTorch model.')

parser.add_argument('--model', type=str, default="models/model.pth")
parser.add_argument('--local', action='store_true')
parser.add_argument('--path', type=str, default="data/img.jpeg")
parser.add_argument('--classes', type=str)

# Setup HyperParameters
MODEL_PATH = parser.parse_args().model
IS_LOCAL = parser.parse_args().local
IMAGE_PATH = parser.parse_args().path
IMAGE_NAME = IMAGE_PATH.split("/")[-1]
CLASSES = parser.parse_args().classes

# Get HyperParameters
HIDDEN_UNITS = int(MODEL_PATH.split("_hu")[0].split("_")[-1])
INPUT_SHAPE = int(MODEL_PATH.split("_cc_")[0].split("_")[-1])
IMG_SIZE = (int(MODEL_PATH.split("_img")[0].split("_")[-1]), int(MODEL_PATH.split("_img")[0].split("_")[-1]))
CLASS_NAMES = CLASSES.split(",") if CLASSES else None
assert CLASS_NAMES != None, "Classes names are required"

# 1. Load in image and convert the tensor values to float32

if IS_LOCAL:
  target_image_path = Path(IMAGE_PATH)
else:
  dataset_path = Path("data/")
  dataset_path.mkdir(parents=True, exist_ok=True)
  target_image_path = dataset_path / IMAGE_NAME

  # Download the image if it doesn't exists
  if not target_image_path.is_file():
    with open(target_image_path, "wb") as f:
      request = requests.get(IMAGE_PATH)
      print(f"Downloading {target_image_path}")
      f.write(request.content)
  else:
    print("The image already exists")

target_image = torchvision.io.read_image(target_image_path).type(torch.float32)

# 2. Divide the image pixel values by 255 to get them between [0, 1]
target_image = target_image / 255.
# 3. Transform if necessary
transform = transforms.Compose([
    transforms.Resize(size=IMG_SIZE)
])
target_image = transform(target_image)

# 4. Create and load a model
model = model_builder.TinyVGG(input_shape=INPUT_SHAPE,
                              hidden_units=HIDDEN_UNITS,
                              output_shape=len(CLASS_NAMES)).to(device)

model = utils.load_model(model=model, model_path=MODEL_PATH)

# 5. Turn on model evaluation mode and inference mode
model.eval()
with torch.inference_mode():
    # Add an extra dimension to the image
    target_image = target_image.unsqueeze(dim=0)

    # Make a prediction on image with an extra dimension and send it to the target device
    target_image_pred = model(target_image.to(device))

# 6. Convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
target_image_pred_probs = torch.softmax(target_image_pred, dim=1)

# 7. Convert prediction probabilities -> prediction labels
target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)

# 8. Plot the image alongside the prediction and prediction probability
plt.imshow(target_image.squeeze().permute(1, 2, 0)) # make sure it's the right size for matplotlib
if CLASS_NAMES:
    title = f"Pred: {CLASS_NAMES[target_image_pred_label.cpu()]} | Prob: {target_image_pred_probs.max().cpu():.3f}"
else:
    title = f"Pred: {target_image_pred_label} | Prob: {target_image_pred_probs.max().cpu():.3f}"
plt.title(title)
plt.axis(False)

save_path = f"results/{IMAGE_NAME.split('.')[0]}_prediction.png"
plt.savefig(save_path)
print(f"[INFO] Prediction done and saved in: {save_path}")

Writing predict.py


## 9. ALL IN

In [ ]:
!python train.py --epochs 5 --batch_size 32 --color_channels 3 --hidden_units 10 --lr 0.001 --img_size 64 --url "https://github.com/mrdbourke/pytorch-deep-learning/raw/refs/heads/main/data/pizza_steak_sushi.zip"

Did not find data/pizza_steak_sushi directory, creating one...
Unzipping pizza_steak_sushi data...
  0% 0/5 [00:00<?, ?it/s] Epoch: 1 | train_loss: 1.1026 | train_acc: 0.2539 | test_loss: 1.0994 | test_acc: 0.2292
 20% 1/5 [00:02<00:08,  2.08s/it] Epoch: 2 | train_loss: 1.1051 | train_acc: 0.3203 | test_loss: 1.1163 | test_acc: 0.1979
 40% 2/5 [00:03<00:05,  1.83s/it] Epoch: 3 | train_loss: 1.0828 | train_acc: 0.4844 | test_loss: 1.0982 | test_acc: 0.3229
 60% 3/5 [00:06<00:04,  2.25s/it] Epoch: 4 | train_loss: 1.0757 | train_acc: 0.4336 | test_loss: 1.0843 | test_acc: 0.3532
 80% 4/5 [00:08<00:02,  2.17s/it] Epoch: 5 | train_loss: 1.0325 | train_acc: 0.6133 | test_loss: 1.0099 | test_acc: 0.5739
100% 5/5 [00:11<00:00,  2.27s/it]
[INFO] Total training time: 11.344 seconds
[INFO] Saving model to: models/TinyVGG_model_5_ep_0.001_lr_32_bs_10_hu_3_cc_64_img#79.pth
[INFO] Saving results to: results/TinyVGG_model_5_ep_0.001_lr_32_bs_10_hu_3_cc_64_img#79.csv


In [ ]:
!python test.py --model "models/TinyVGG_model_5_ep_0.001_lr_32_bs_10_hu_3_cc_64_img#79.pth" --url "https://github.com/mrdbourke/pytorch-deep-learning/raw/refs/heads/main/data/pizza_steak_sushi.zip"

data/pizza_steak_sushi directory exists.
Unzipping pizza_steak_sushi data...
[INFO] Loading model from: models/TinyVGG_model_5_ep_0.001_lr_32_bs_10_hu_3_cc_64_img#79.pth
100% 3/3 [00:01<00:00,  2.85it/s]
[INFO] Model: TinyVGG
[INFO] Model loss: 1.0099
[INFO] Model accuracy: 57.3864%
[INFO] Model execution time: 1.083 seconds


In [ ]:
!python predict.py --model "models/TinyVGG_model_5_ep_0.001_lr_32_bs_10_hu_3_cc_64_img#79.pth" --path "https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/images/04-pizza-dad.jpeg" --classes "pizza,steak,sushi"

The image already exists
[INFO] Loading model from: models/TinyVGG_model_5_ep_0.001_lr_32_bs_10_hu_3_cc_64_img#79.pth
[INFO] Prediction done and saved in: results/04-pizza-dad_prediction.png
